In [1]:
import cv2
import numpy as np
from collections import deque

# --- 1. Configuración Inicial ---

# Estructura de datos para almacenar los puntos de dibujo
# Usamos un deque para cada color que queremos trazar
# Formato: {color_name: [deque_de_puntos, (B, G, R)_color_linea]}
points_data = {
    "azul": [deque(maxlen=1024), (255, 0, 0)],
    # Puedes añadir más colores aquí si quieres
    # "verde": [deque(maxlen=1024), (0, 255, 0)] 
}

# Color que actuará como borrador
eraser_color_name = "rosa"

# Definimos los rangos de color en HSV
# ¡ESTOS VALORES PUEDEN NECESITAR AJUSTE!
# Usa un script de "Color Picker" para encontrar los valores para tu objeto/luz
color_ranges = {
    "azul": ([100, 100, 100], [120, 255, 255]),
    "rosa": ([140, 120, 150], [170, 255, 255]) 
}

# Índice para el color de dibujo actual
current_color_index = 0
color_names = list(points_data.keys())

# --- 2. Bucle Principal de Video ---

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: No se pudo abrir la cámara.")
    exit()

print("Pizarra Virtual iniciada. Mueve un objeto azul para dibujar.")
print("Mueve un objeto rosa para borrar. Presiona 'c' para limpiar todo.")
print("Presiona 'q' para salir.")

# Creamos una ventana para dibujar
paint_window = None

while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    frame = cv2.flip(frame, 1)
    
    # Si es el primer frame, inicializamos la pizarra
    if paint_window is None:
        paint_window = np.zeros_like(frame)

    # Convertimos a HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # --- 3. Detección y Lógica ---

    # Iteramos sobre los colores de dibujo y el borrador
    for color_name, (lower, upper) in color_ranges.items():
        # Creamos la máscara y la limpiamos
        mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
        mask = cv2.erode(mask, None, iterations=2)
        mask = cv2.dilate(mask, None, iterations=2)

        # Encontramos contornos
        contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        center = None
        
        # Si se encuentra un contorno
        if len(contours) > 0:
            # Encontramos el contorno más grande
            c = max(contours, key=cv2.contourArea)
            ((x, y), radius) = cv2.minEnclosingCircle(c)
            
            # Solo consideramos contornos de un tamaño razonable
            if radius > 10:
                # Dibujamos un círculo alrededor del objeto detectado en el frame original
                cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
                # Calculamos el centro
                M = cv2.moments(c)
                center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

                # Lógica de borrado
                if color_name == eraser_color_name:
                    # Si el borrador está activo, borramos los puntos cercanos
                    for color_key, (points_deque, _) in points_data.items():
                        # Hacemos una copia para poder modificar mientras iteramos
                        temp_points = list(points_deque)
                        for i, point in enumerate(temp_points):
                            if point is not None and np.linalg.norm(np.array(point) - np.array(center)) < radius * 1.5:
                                # Borra el punto y los cercanos para un borrado más suave
                                for j in range(max(0, i-5), min(len(temp_points), i+5)):
                                     if j < len(points_deque):
                                        points_deque[j] = None
                else:
                    # Lógica de dibujo
                    if color_name == color_names[current_color_index]:
                        points_data[color_name][0].appendleft(center)

    # --- 4. Dibujar la Trayectoria ---
    # Limpiamos la pizarra para redibujar
    paint_window.fill(0)
    for color_key, (points_deque, line_color) in points_data.items():
        for i in range(1, len(points_deque)):
            if points_deque[i - 1] is None or points_deque[i] is None:
                continue
            # Dibujamos una línea en nuestra ventana de pintura
            cv2.line(paint_window, points_deque[i - 1], points_deque[i], line_color, 5)

    # --- 5. Combinar y Mostrar ---
    # Combinamos el frame de la cámara con la ventana de pintura
    frame = cv2.add(frame, paint_window)
    cv2.imshow("Pizarra Virtual", frame)
    # cv2.imshow("Pizarra", paint_window) # Descomenta para ver solo el dibujo

    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
    # Limpiar toda la pantalla con la tecla 'c'
    elif key == ord("c"):
        for color_key in points_data:
            points_data[color_key][0].clear()
        paint_window.fill(0)

# --- 6. Liberar Recursos ---
cap.release()
cv2.destroyAllWindows()

Pizarra Virtual iniciada. Mueve un objeto azul para dibujar.
Mueve un objeto rosa para borrar. Presiona 'c' para limpiar todo.
Presiona 'q' para salir.
